# nunjucks


In [ ]:
from lovely_docs.settings import settings, GitSource
from lovely_docs.git import clone_repo
from lovely_docs.docs import (
    build_markdown_doc_tree,
    process_tree_depth_first,
    calculate_total_usage,
    calculate_cost,
    save_processed_documents,
)

from pathlib import Path
from datetime import datetime
import logging


In [ ]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)


In [ ]:
source = GitSource(
    name="nunjucks",
    doc_dir="docs",
    repo="https://github.com/mozilla/nunjucks",
    commit="master",
    ecosystems=["js"],
)

commit, clone_dir = clone_repo(source)
source.commit = commit


In [ ]:
run_id = datetime.now().strftime("%Y%m%d-%H%M%S")
virtual_root = settings.project_root / "_tmp_docs" / "nunjucks" / run_id

(virtual_root / "nunjucks").mkdir(parents=True, exist_ok=True)

doc_files = [
    "api.md",
    "getting-started.md",
    "templating.md",
]

missing = []
for fname in doc_files:
    src = clone_dir / "docs" / fname
    if not src.exists():
        missing.append(str(src))
        continue
    (virtual_root / "nunjucks" / fname).write_text(src.read_text())

assert not missing, f"Missing doc files: {missing}"


In [ ]:
tree = build_markdown_doc_tree(virtual_root, extensions=[".md"])


In [ ]:
processed_tree = await process_tree_depth_first(settings, tree, source.name)

+0.842s DEBUG preprocessor/nbs/lovely_docs/docs.py:156 llm_process_page Processing nunjucks/api.md
+0.338s DEBUG preprocessor/nbs/lovely_docs/docs.py:156 llm_process_page Processing nunjucks/getting-started.md
+0.017s DEBUG preprocessor/nbs/lovely_docs/docs.py:156 llm_process_page Processing nunjucks/templating.md
+47.372s DEBUG preprocessor/nbs/lovely_docs/docs.py:258 llm_process_directory Processing nunjucks


In [ ]:
usage = calculate_total_usage(processed_tree)
cost, input_cost, output_cost = calculate_cost(usage, 1, 5)

print(f"\nTotal Usage:")
print(f"  Input tokens: {usage.input:,}")
print(f"  Output tokens: {usage.output:,}")
print(f"  Total tokens: {(usage.input + usage.output):,}")
print(f"Cost:")
print(f"  Total: ${cost:.2f}")
print(f"  Input: ${input_cost:.2f}")
print(f"  Output: ${output_cost:.2f}")



Total Usage:
  Input tokens: 36,289
  Output tokens: 13,647
  Total tokens: 49,936
Cost:
  Total: $0.10
  Input: $0.04
  Output: $0.07


In [ ]:
# processed_tree.children[0].name = "nunjucks"
# processed_tree.children[0].displayName = "Nunjucks"

In [ ]:
save_processed_documents(
    source, settings.output_dir / source.name.replace("/", "_"), processed_tree
)